# Model Training - FastText

In [1]:
# Import required libraries
import numpy as np
import pandas as pd
import os 
from gensim.models import FastText
from sklearn.metrics.pairwise import cosine_similarity
import mlflow
import json

import warnings
warnings.filterwarnings("ignore")

In [254]:
import yaml

def load_params():
    with open("../params.yaml") as f:
        params = yaml.safe_load(f)
    return params

params = load_params()

In [3]:
data = pd.read_pickle("../"+params["model_pipeline"]["recipe_path"])

## Model Training

In [5]:
# fasttext_model = FastText(data.ingredients, **params["model_pipeline"]["fast_text"])
# fasttext_model.save(params["model_pipeline"]["model_path"])
fasttext_model = FastText.load("../" + params["model_pipeline"]["model_path"])

In [78]:
# Quick sanity check
fasttext_model.wv.most_similar("chicken", topn=10)

[('chicken_tender', 0.8974050283432007),
 ('chicken_leg_quarter', 0.8684288859367371),
 ('chicken_wing', 0.8680126667022705),
 ('chicken_tenderloin', 0.8668988347053528),
 ('chicken_breast', 0.8451071977615356),
 ('chicken_thigh', 0.8280680775642395),
 ('chicken_breast/thigh', 0.8230547308921814),
 ('chicken_breast/thighs', 0.8133108019828796),
 ('chicken_drumstick', 0.7985769510269165),
 ('chicken_bouillon', 0.7671944499015808)]

### IDF (Inverse Document Frequency)

In [259]:
idf = {}
# Calculate IDF
for vocab in fasttext_model.wv.index_to_key:
    n_recipe_contains_vocab = data.ingredients.apply(lambda x: vocab in x).sum()
    idf[vocab] = np.log(len(data.ingredients) / (n_recipe_contains_vocab + 1e-9))

idf

{'salt': 0.43756323278446185,
 'garlic': 0.9736349773773009,
 'water': 1.3433086441537483,
 'sugar': 1.5919917083637571,
 'soy': 1.6608262141840013,
 'black_pepper': 1.695465198807162,
 'egg': 1.7484856093407433,
 'ginger': 1.8550953443989042,
 'olive_oil': 1.9424564328800327,
 'onion': 2.1313050156881426,
 'vanilla_extract': 2.1806656847698176,
 'pepper': 2.367524817746226,
 'unsalted_butter': 2.3804490687271405,
 'lemon': 2.3984971385988136,
 'shaoxing_wine': 2.405141681317471,
 'baking': 2.405141681317471,
 'milk': 2.4151920171709556,
 'cornstarch': 2.4424974678611764,
 'sesame_oil': 2.4599499178123723,
 'carrot': 2.5405796522123336,
 'white_pepper': 2.565701746329623,
 'cinnamon': 2.5854652354484235,
 'maple_syrup': 2.5874632381110922,
 'scallion': 2.613807213450641,
 'vegetable_oil': 2.642975948041237,
 'butter': 2.66434428244689,
 'green_onion': 2.717571923457835,
 'brown_sugar': 2.7244290931839563,
 'all-purpose_flour': 2.7737928163021763,
 'baking_soda': 2.8131099196429146,
 'c

### Ingredient Vectors

In [249]:
# Prepare ingredient vectors
ingredient_vectors = []
for ingredients in data.ingredients:
    # Get the normalized vectors for the recipe ingredients and weight by IDF
    embedding_vecs = [fasttext_model.wv.get_vector(ing, norm=True) * idf.get(ing, 1) for ing in ingredients]
    ingredient_vectors.append(embedding_vecs)

ingredient_vector_means = np.array([np.mean(ing_vector, axis=0) for ing_vector in ingredient_vectors])

### Title Vectors

In [247]:
# Prepare title vectors
title_vectors = []
for title in data.recipe_title:
    # Get the vector for the recipe title and normalize it
    title_split = title.split()
    embedding_vecs = [fasttext_model.wv.get_vector(title, norm=True) for title in title_split]
    title_vectors.append(embedding_vecs)

title_vector_means = np.array([np.mean(ing_vector, axis=0) for ing_vector in title_vectors])

### Query Processing

In [257]:
# Query processing with IDF weighting
text = "rice, duck" # Example input
query = [ingredient.strip() for ingredient in text.split(",") if ingredient]
print("query:", query)

# Get the normalized vectors for the search query
query_vecs = np.array([fasttext_model.wv.get_vector(input, norm=True) for input in query])

# Borrow IDF from top 3 most similar ingredients
idf_top_weights = params["model_pipeline"]["model_scoring"]["idf_top_weights"] # Weights for top 3 similar ingredients

query_idfs = []
for query in query_vecs:
    similar_ings = [ing for ing, _ in fasttext_model.wv.similar_by_vector(query, topn=3)]
    ing_idf = np.sum([weight * idf.get(similar_ing, 1) for similar_ing, weight in zip(similar_ings, idf_top_weights)])
    query_idfs.append(ing_idf)

# Apply IDF 
query_vecs = np.array([(query_vec * query_idf) for query_vec, query_idf in zip(query_vecs, query_idfs)])

query: ['rice', 'duck']


### Model Scoring System

In [258]:
# Prediction scoring system
# Calculate MaxSim for each recipe in the data
# Might be slow for large amount of data
ingredient_max_sim = np.array([])
for ing_vector in ingredient_vectors:
    tokens_sim = cosine_similarity(query_vecs, ing_vector)
    
    a_best = tokens_sim.max(axis=1)
    b_best = tokens_sim.max(axis=0)
    
    score = 0.5 * (a_best.mean() + b_best.mean())
    ingredient_max_sim = np.append(ingredient_max_sim, score)

# Calculate cosine similarity between the mean query vector and the mean ingredients vector
mean_query_vec = query_vecs.mean(axis=0)
ingredient_sim = cosine_similarity(mean_query_vec.reshape(1, -1), ingredient_vector_means)
title_sim = cosine_similarity(mean_query_vec.reshape(1, -1), title_vector_means)

score = (params["model_pipeline"]["model_scoring"]["w_cosine"] * ingredient_sim[0] + 
            params["model_pipeline"]["model_scoring"]["w_maxsim"] * ingredient_max_sim + 
            params["model_pipeline"]["model_scoring"]["w_title"] * title_sim[0])

rec_idx = np.argsort(score)[-params["model_service"]["n_recs"]:][::-1]
# rec_idx = [4557, 5297, 3977, 6109, 366, 3572, 6262, 314]
data.iloc[rec_idx]

,recipe_title,recipe_url,ingredients,img_url,num_steps,prep_time,cook_time,total_time,serving_size,calories,...,fiber,sugar,calcium,iron,custom_time,vitamin_c,vitamin_a,trans_fat,polyunsaturated_fat,monounsaturated_fat
2145,Sour Plum Duck A Hakka Chinese Classic,https://thewoksoflife.com/sour-plum-duck/,"[duck, salt, white_pepper, ginger, canola_oil,...",https://thewoksoflife.com/wp-content/uploads/2...,8.0,35.0,165.0,200.0,0.0,765.0,...,3.0,11.0,52.0,4.8,0.0,16.3,475.0,0.00,0.0,NaN
5066,Jasmine Rice,https://www.recipetineats.com/how-to-cook-jasm...,"[jasmine_rice, water]",https://www.recipetineats.com/tachyon/2020/06/...,7.0,1.0,12.0,23.0,0.0,169.0,...,1.0,1.0,13.0,1.0,10.0,0.0,0.0,0.00,0.0,NaN
2046,Steamed Pork With Rice Powder,https://thewoksoflife.com/steamed-pork-rice-po...,"[pork_belly, ginger, shaoxing_wine, fermented_...",https://thewoksoflife.com/wp-content/uploads/2...,5.0,90.0,90.0,180.0,0.0,625.0,...,4.0,1.0,127.0,2.2,0.0,4.1,75.0,0.00,0.0,NaN
5045,Brown Rice,https://www.recipetineats.com/how-to-cook-brow...,"[brown_rice_–_grain_grain/basmati, water, shor...",https://www.recipetineats.com/tachyon/2020/09/...,10.0,1.0,30.0,41.0,0.0,229.0,...,2.0,0.0,26.0,1.0,10.0,0.0,0.0,0.00,0.0,NaN
2344,Sticky Rice Hashbrowns,https://thewoksoflife.com/rice-hashbrowns/,"[rice, short_grain_white_rice, water, pork, so...",https://thewoksoflife.com/wp-content/uploads/2...,7.0,120.0,45.0,165.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,NaN
2255,Roasted Braised Duck,https://thewoksoflife.com/roasted-braised-duck/,"[duck, oil, ginger, garlic, rock_sugar, chines...",https://thewoksoflife.com/wp-content/uploads/2...,7.0,15.0,75.0,90.0,0.0,553.0,...,0.4,2.0,32.0,4.0,0.0,5.0,220.0,0.01,7.0,25.0
1925,One Pot Rice Cooker Rice With Dace Fish,https://thewoksoflife.com/dace-fish-black-bean...,"[rice, dace_fish_salted_black_bean, carrot, pe...",https://thewoksoflife.com/wp-content/uploads/2...,4.0,5.0,25.0,30.0,0.0,551.0,...,8.0,2.0,36.0,1.1,0.0,8.7,2840.0,0.00,0.0,NaN
1737,Roasted Chicken With Sticky Rice,https://thewoksoflife.com/chicken-with-sticky-...,"[shiitake_mushroom, chicken_thigh, onion, garl...",https://thewoksoflife.com/wp-content/uploads/2...,7.0,150.0,60.0,210.0,0.0,603.0,...,3.0,2.0,27.0,3.0,0.0,2.0,119.0,0.00,0.0,NaN
2074,Braised Duck With Taro,https://thewoksoflife.com/braised-duck-taro/,"[duck, taro, oil, piece_rock_sugar, ginger, ga...",https://thewoksoflife.com/wp-content/uploads/2...,4.0,30.0,50.0,80.0,0.0,805.0,...,4.0,1.0,63.0,4.6,0.0,9.8,370.0,0.00,0.0,NaN
1617,Chinese Sausage Rice Cooker Rice,https://thewoksoflife.com/chinese-sausage-rice...,"[jasmine_rice, water, lap_cheong]",https://thewoksoflife.com/wp-content/uploads/2...,5.0,5.0,20.0,25.0,0.0,395.0,...,1.0,5.0,23.0,1.0,0.0,0.0,0.0,0.00,1.0,1.0


## Wrap Model with MLFlow Python Model

Models from Code (Add set model) or Load as Class (Restart kernel after every overwrite)

In [ ]:
class RecipeFastText(mlflow.pyfunc.PythonModel):
    def __init__(self, params):
        self.params = params

    def load_context(self, context):
        self.model = FastText.load(context.artifacts["model_path"])
        self.data = pd.read_pickle(context.artifacts["data_path"])

        # Prepare IDF
        self.idf = {}
        for vocab in self.model.wv.index_to_key:
            n_recipe_contains_vocab = self.data.ingredients.apply(lambda x: vocab in x).sum()
            self.idf[vocab] = np.log(len(self.data.ingredients) / (n_recipe_contains_vocab + 1e-9))

        # Prepare ingredient vectors
        self.ingredient_vectors = []
        for ingredients in self.data.ingredients:
            # Get the vectors for the recipe ingredients and normalize them
            embedding_vecs = [self.model.wv.get_vector(ing, norm=True) * self.idf.get(ing, 1)  for ing in ingredients]
            self.ingredient_vectors.append(embedding_vecs)

        self.ingredient_vector_means = np.array([np.mean(ing_vector, axis=0) for ing_vector in self.ingredient_vectors])

        # Prepare title vectors
        self.title_vectors = []
        for title in self.data.recipe_title:
            # Get the vector for the recipe title and normalize it
            title_split = title.split()
            embedding_vecs = [self.model.wv.get_vector(title, norm=True) for title in title_split]
            self.title_vectors.append(embedding_vecs)

        self.title_vector_means = np.array([np.mean(ing_vector, axis=0) for ing_vector in self.title_vectors])

    def predict(self, model_input: list[str]) -> list[float]:
        """Predicts the recipe similarity score based on the input string."""
        
        # Get the vectors for the search query and normalize them
        query_vecs = np.array([self.model.wv.get_vector(input, norm=True) for input in model_input])

        # Borrow IDF from top 3 most similar ingredients
        idf_top_weights = self.params["model_scoring"]["idf_top_weights"] # Weights for top 3 similar ingredients

        query_idfs = []
        for query in query_vecs:
            similar_ings = [ing for ing, _ in self.model.wv.similar_by_vector(query, topn=3)]
            ing_idf = np.sum([weight * self.idf.get(similar_ing, 1) for similar_ing, weight in zip(similar_ings, idf_top_weights)])
            query_idfs.append(ing_idf)

        # Apply IDF 
        query_vecs = np.array([(query_vec * query_idf) for query_vec, query_idf in zip(query_vecs, query_idfs)])

        # Calculate MaxSim for each recipe in the data
        # Might be slow for large amount of data
        ingredient_max_sim = np.array([])
        for ing_vector in self.ingredient_vectors:
            tokens_sim = cosine_similarity(query_vecs, ing_vector)

            a_best = tokens_sim.max(axis=1)
            b_best = tokens_sim.max(axis=0)
            
            score = 0.5 * (a_best.mean() + b_best.mean())
            ingredient_max_sim = np.append(ingredient_max_sim, score)

        # Calculate cosine similarity between the mean query vector and the mean ingredients vector
        mean_query_vec = query_vecs.mean(axis=0)
        ingredient_sim = cosine_similarity(mean_query_vec.reshape(1, -1), self.ingredient_vector_means)
        title_sim = cosine_similarity(mean_query_vec.reshape(1, -1), self.title_vector_means)

        score = (self.params["model_scoring"]["w_cosine"] * ingredient_sim[0] +
                self.params["model_scoring"]["w_maxsim"] * ingredient_max_sim +
                self.params["model_scoring"]["w_title"] * title_sim[0])

        return score

### Log Model to MLFlow

In [111]:
mlflow.set_tracking_uri("http://127.0.0.1:8080")
mlflow.set_experiment("FastText-Model")

2025/08/30 18:07:49 INFO mlflow.tracking.fluent: Experiment with name 'FastText-Model' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/300877920446874715', creation_time=1756602469576, experiment_id='300877920446874715', last_update_time=1756602469576, lifecycle_stage='active', name='FastText-Model', tags={}>

In [ ]:
# Load params
params = load_params()

with mlflow.start_run():
    # Load data
    data = pd.read_pickle(params["model_pipeline"]["recipe_path"])

    # Load fasttext model
    fasttext_model = RecipeFastText(params["model_pipeline"])

    # Log params
    mlflow.log_params(params["model_pipeline"]["fast_text"])
    mlflow.log_params(params["model_pipeline"]["model_scoring"])

    # Log model
    model_info = mlflow.pyfunc.log_model(
        name="fasttext_model",
        python_model=fasttext_model,
        artifacts={"model_path": params["model_pipeline"]["model_path"],
                   "model_ngram_path": params["model_pipeline"]["model_ngram_path"],
                   "data_path": params["model_pipeline"]["recipe_path"]},
        pip_requirements=["gensim==4.3.3"]
    )

    # Log local URI
    local_uri = os.path.join("../mlflow/mlartifacts", *model_info.artifact_path.split("/")[1:])
    mlflow.log_params({"local_uri": local_uri})
    
    # Log metadata
    metadata = {
        "server_uri": model_info.model_uri,
        "local_uri": local_uri,
        "run_id": model_info.run_id,
        "time_logged": model_info.utc_time_created,
        "model_params": params["model_pipeline"]
    }

    # Save metadata locally
    metadata["model_params"] = params["model_pipeline"]
    with open("../training_metadata.json", "w") as f:
        f.write(json.dumps(metadata, indent=4))

    mlflow.log_artifact("../training_metadata.json")


🏃 View run serious-stoat-300 at: http://127.0.0.1:8080/#/experiments/300877920446874715/runs/32cc39d3e1e24799886476ad91037e36
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/300877920446874715


### Quick Sanity Check

In [ ]:
with open("../training_metadata.json", "r") as f:
    metadata = json.load(f)

model = mlflow.pyfunc.load_model(metadata["local_uri"])

In [42]:
sim_scores = model.predict(["bread"])  # Example usage
rec_idx = np.argsort(sim_scores)[::-1][:10] # Get top recommendations
data.iloc[rec_idx]

,id,recipe_title,recipe_url,ingredients,num_steps,total_time,prep_time,cook_time,custom_time,calories,...,sodium,potassium,fiber,sugar,vitamin_a,vitamin_c,calcium,iron,serving_size,image_url
3974,3974,Breakfast Blt Sandwich,https://natashaskitchen.com/breakfast-blt-sand...,"[bread, bacon, egg, avocado, tomato, green_let...",6.0,35.0,15.0,20.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,https://natashaskitchen.com/wp-content/uploads...
5708,5708,Bacon In A Wok After School Sandwiches,https://thewoksoflife.com/bacon-in-a-wok-sandw...,"[bacon, crusty_bread]",6.0,10.0,0.0,10.0,0.0,645.0,...,1094.0,295.0,3.00,3.0,0.0,0.0,56.0,5.00,0.0,https://thewoksoflife.com/wp-content/uploads/2...
3943,3943,Manchego Cheese Bites,https://natashaskitchen.com/manchego-cheese-bi...,"[manchego_cheese, fig_spread, prosciutto]",3.0,5.0,5.0,0.0,0.0,83.0,...,110.0,9.0,0.03,1.0,79.0,0.2,152.0,0.03,0.0,https://natashaskitchen.com/wp-content/uploads...
4679,4679,Grilled Garlic Bread,https://www.recipetineats.com/grilled-garlic-b...,"[bread_loaf, stick_salted_butter, garlic, pars...",9.0,15.0,5.0,10.0,0.0,331.0,...,495.0,127.0,2.00,4.0,575.0,1.3,100.0,2.40,0.0,https://www.recipetineats.com/tachyon/2019/06/...
3600,3600,Open Faced Avocado Spread Sandwiches,https://natashaskitchen.com/open-faced-avocado...,"[bread, avocado, salt, squeezed_lemon, black_p...",4.0,10.0,7.0,3.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,https://natashaskitchen.com/wp-content/uploads...
5201,5201,Caesar Salad Wonton Cups,https://www.recipetineats.com/caesar-salad-won...,"[wonton_wrapper, romaine_lettuce, chicken, bac...",7.0,30.0,20.0,10.0,0.0,117.0,...,338.0,72.0,0.00,0.9,100.0,2.5,50.0,0.90,54.0,https://www.recipetineats.com/tachyon/2014/08/...
2603,2603,My Go-To Avocado Toast,https://minimalistbaker.com/my-go-to-avocado-t...,"[bread, avocado, vegan_parmesan, red_pepper_fl...",1.0,5.0,2.0,3.0,0.0,293.0,...,386.0,415.0,6.20,3.6,100.0,8.3,30.0,1.30,1.0,https://minimalistbaker.com/wp-content/uploads...
986,986,Garlic Bread,https://www.loveandlemons.com/garlic-bread/,"[unsalted_butter, parsley, garlic, salt, frenc...",4.0,20.0,10.0,10.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,https://cdn.loveandlemons.com/wp-content/uploa...
3918,3918,Salami Cream Cheese Sandwich,https://natashaskitchen.com/salami-cream-chees...,"[cream_cheese, green_onion, dill, garlic, sand...",3.0,5.0,5.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,https://natashaskitchen.com/wp-content/uploads...
5081,5081,Garlic Butter Cauliflower Pasta With Pangrattato,https://www.recipetineats.com/garlic-butter-ca...,"[bread, oil_spray, unsalted_butter, olive_oil,...",10.0,25.0,10.0,15.0,0.0,408.0,...,148.0,694.0,4.70,4.9,300.0,108.9,60.0,3.80,288.0,https://www.recipetineats.com/tachyon/2015/08/...
